You have to work on the (NYC Trip Fare)[https://www.kaggle.com/api/v1/datasets/download/diishasiing/revenue-for-cab-drivers/archive.zip] repository. You can skip the store_and_fwd_flag column, but it’s a bonus point if you can manage it correctly.

Notes
1. It is mandatory to use GitHub for developing the project.
2. The project must be a jupyter notebook.
3. There is no restriction on the libraries that can be used, nor on the Python version.
4. All questions on the project must be asked in the Discussion forum on the course website.
5. At most 3 students can be in each group. You must create the groups by yourself. You can use the Discussion forum to create the groups.
6. You do not have to send me the project before the discussion.
7. You do not have to prepare any slides for the discussion.

In [1]:
# Import section
import pandas as pd
import numpy as np

In [ ]:
# To manage the issue with the store_and_fwd_flag column we replaced all the NaN values with Unknown so that the column
# won't have mixed types.
df = pd.read_csv('data.csv', dtype={'store_and_fwd_flag': str})
df["store_and_fwd_flag"] = df["store_and_fwd_flag"].replace({pd.NA: 'Unknown'}, inplace=True)


# Converting the values of these two columns to_datetime will help us later on with the tasks, especially with task #7.
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

C:\Users\1\AppData\Local\Temp\ipykernel_18980\1224477848.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["store_and_fwd_flag"].replace({pd.NA: 'Unknown'}, inplace=True)


1. Extract all trips with trip_distance larger than 50

In [3]:
# We filter the dataframe to get the desired result, showing only the first 4 rows for clarity. The .head() method
# will occur often in the code for this reason.
df[df["trip_distance"] > 50].head(4)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
23842,2.0,2020-01-01 01:53:07,2020-01-01 03:54:41,1.0,52.30,5.0,N,262,265,1.0,300.0,0.0,0.0,61.78,6.12,0.3,370.70,2.5
39013,2.0,2020-01-01 02:05:07,2020-01-01 03:03:10,1.0,51.23,5.0,N,264,264,1.0,329.0,0.0,0.5,100.78,6.12,0.3,436.70,0.0
41620,1.0,2020-01-01 03:05:54,2020-01-01 04:16:26,1.0,53.80,5.0,N,132,265,1.0,250.0,0.0,0.0,53.35,16.62,0.3,320.27,0.0
58262,2.0,2020-01-01 05:36:12,2020-01-01 06:40:06,1.0,55.23,5.0,N,132,265,2.0,170.0,0.0,0.5,0.00,18.26,0.3,189.06,0.0


2. Extract all trips where payment_type is missing


In [4]:
# If the payment type is missing this means its value will be NaN, so we use the .isna() method to select the trips
# with the NaN payment type.
df[df["payment_type"].isna()].head(4)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
6339567,NaN,2020-01-01 08:51:00,2020-01-01 09:19:00,NaN,13.69,NaN,Unknown,136,232,NaN,51.05,2.75,0.5,0.0,0.0,0.3,54.60,0.0
6339568,NaN,2020-01-01 08:38:43,2020-01-01 08:51:08,NaN,3.42,NaN,Unknown,121,9,NaN,27.06,2.75,0.0,0.0,0.0,0.3,30.11,0.0
6339569,NaN,2020-01-01 08:27:00,2020-01-01 08:32:00,NaN,2.20,NaN,Unknown,197,216,NaN,24.36,2.75,0.5,0.0,0.0,0.3,27.91,0.0
6339570,NaN,2020-01-01 08:46:00,2020-01-01 08:57:00,NaN,0.84,NaN,Unknown,262,236,NaN,26.08,2.75,0.5,0.0,0.0,0.3,29.63,0.0


3. For each (PULocationID, DOLocationID) pair, determine the number of trips

In [5]:
# .groupby() groups the data in df by the pickup (PULocationID) and drop-off (DOLocationID) locations, and then 
# counts the number of trips for each unique pair of locations using .size()
trip_counts = df.groupby(['PULocationID', 'DOLocationID']).size()
trip_counts

PULocationID  DOLocationID
1             1                638
              50                 1
              68                 1
              138                2
              140                1
                              ... 
265           259                2
              261                1
              263                4
              264              317
              265             2508
Length: 31277, dtype: int64

4. Save all rows with missing VendorID, passenger_count, store_and_fwd_flag, payment_type in a new dataframe called bad, and remove those rows from the original dataframe.

In [6]:
# Since it's not clear wether VendorID, passenger_count, store_and_fwd_flag, payment_type must ALL be missing for that row
# to be consired 'bad' or it is enough to have rows where ANY of these are NaN, the latter is what we'll opt for.
# We use the .isna() like in task 2
bad = df[df[['VendorID', 'passenger_count', 'store_and_fwd_flag', 'payment_type']].isna().any(axis=1)]
bad

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
6339567,NaN,2020-01-01 08:51:00,2020-01-01 09:19:00,NaN,13.69,NaN,Unknown,136,232,NaN,51.05,2.75,0.5,0.0,0.00,0.3,54.60,0.0
6339568,NaN,2020-01-01 08:38:43,2020-01-01 08:51:08,NaN,3.42,NaN,Unknown,121,9,NaN,27.06,2.75,0.0,0.0,0.00,0.3,30.11,0.0
6339569,NaN,2020-01-01 08:27:00,2020-01-01 08:32:00,NaN,2.20,NaN,Unknown,197,216,NaN,24.36,2.75,0.5,0.0,0.00,0.3,27.91,0.0
6339570,NaN,2020-01-01 08:46:00,2020-01-01 08:57:00,NaN,0.84,NaN,Unknown,262,236,NaN,26.08,2.75,0.5,0.0,0.00,0.3,29.63,0.0
6339571,NaN,2020-01-01 08:21:00,2020-01-01 08:38:00,NaN,7.24,NaN,Unknown,45,142,NaN,25.28,2.75,0.5,0.0,0.00,0.3,28.83,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405003,NaN,2020-01-31 22:51:00,2020-01-31 23:22:00,NaN,3.24,NaN,Unknown,237,234,NaN,17.59,2.75,0.5,0.0,0.00,0.3,21.14,0.0
6405004,NaN,2020-01-31 22:10:00,2020-01-31 23:26:00,NaN,22.13,NaN,Unknown,259,45,NaN,46.67,2.75,0.5,0.0,12.24,0.3,62.46,0.0
6405005,NaN,2020-01-31 22:50:07,2020-01-31 23:17:57,NaN,10.51,NaN,Unknown,137,169,NaN,48.85,2.75,0.0,0.0,0.00,0.3,51.90,0.0
6405006,NaN,2020-01-31 22:25:53,2020-01-31 22:48:32,NaN,5.49,NaN,Unknown,50,42,NaN,27.17,2.75,0.0,0.0,0.00,0.3,30.22,0.0


In [7]:
# To remove the 'bad' rows from the dataframe we use .drop()
df = df.drop(bad.index)
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339562,2.0,2020-01-31 23:38:07,2020-01-31 23:52:21,1.0,2.10,1.0,N,163,246,1.0,11.0,0.5,0.5,2.96,0.0,0.3,17.76,2.5
6339563,2.0,2020-01-31 23:00:18,2020-01-31 23:19:18,1.0,2.13,1.0,N,164,79,1.0,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5
6339564,2.0,2020-01-31 23:24:22,2020-01-31 23:40:39,1.0,2.55,1.0,N,79,68,1.0,12.5,0.5,0.5,3.26,0.0,0.3,19.56,2.5
6339565,2.0,2020-01-31 23:44:22,2020-01-31 23:54:00,1.0,1.61,1.0,N,100,142,2.0,8.5,0.5,0.5,0.00,0.0,0.3,12.30,2.5


5. Add a duration column storing how long each trip has taken (use tpep_pickup_datetime, tpep_dropoff_datetime)

In [8]:
# To determine how long each trip has taken we simply compute the difference between tpep_dropoff_datetime and tpep_pickup_datetime
# and we store the results in a new column called 'duration'
df['duration'] = (pd.to_datetime(df['tpep_dropoff_datetime']) - pd.to_datetime(df['tpep_pickup_datetime']))
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,0 days 00:04:48
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5,0 days 00:07:25
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5,0 days 00:06:11
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0,0 days 00:04:51
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0,0 days 00:02:18


6. For each pickup location, determine how many trips have started there.


In [9]:
# To complete this task, we need to group by the PULocationID and then count how many trips exist for each unique group 
df.groupby(df["PULocationID"]).size()

PULocationID
1         753
2           3
3          70
4        9902
5          39
        ...  
261     34229
262     85591
263    123997
264     43779
265      3090
Length: 260, dtype: int64

7. Cluster the pickup time of the day into 30-minute intervals (e.g. from 02:00 to 02:30)


In [10]:
# This function takes a datetime object `dt` as input and it returns a formatted 30-minute interval string based on 
# the input in the format "HH:MM - HH:MM" in which it falls.
# - If the minute of `dt` is less than 30, the interval will be from "HH:00" to "HH:30".
# - If the minute of `dt` is 30 or greater, the interval will be from "HH:30" to "(HH+1):00". 
def get_30min_interval(dt):

    hour = dt.hour
    minute = 0 if dt.minute < 30 else 30    
    
    if (minute == 30):
        return f"{hour:02}:{minute:02} - {hour+1:02}:{minute-30:02}"
        
    return f"{hour:02}:{minute:02} - {hour:02}:{minute+30:02}"

In [11]:
# To cluster the pickup time into 30-minutes intervals we apply the get_30min_interval function we created below to the df column.
df['pickup_interval'] = df['tpep_pickup_datetime'].apply(get_30min_interval)

df[['tpep_pickup_datetime', 'pickup_interval']]

,tpep_pickup_datetime,pickup_interval
0,2020-01-01 00:28:15,00:00 - 00:30
1,2020-01-01 00:35:39,00:30 - 01:00
2,2020-01-01 00:47:41,00:30 - 01:00
3,2020-01-01 00:55:23,00:30 - 01:00
4,2020-01-01 00:01:58,00:00 - 00:30
...,...,...
6339562,2020-01-31 23:38:07,23:30 - 24:00
6339563,2020-01-31 23:00:18,23:00 - 23:30
6339564,2020-01-31 23:24:22,23:00 - 23:30
6339565,2020-01-31 23:44:22,23:30 - 24:00


8. For each interval, determine the average number of passengers and the average fare amount.


In [12]:
# To determine that we calculate the average value of passenger_count and fare_amount using .mean()
df.groupby('pickup_interval')[['passenger_count', 'fare_amount']].mean().head(5)

,passenger_count,fare_amount
pickup_interval,,
00:00 - 00:30,1.572848,13.526433
00:30 - 01:00,1.584345,13.214132
01:00 - 01:30,1.578933,12.699554
01:30 - 02:00,1.589182,12.265997
02:00 - 02:30,1.587479,12.089669


9. For each payment type and each interval, determine the average fare amount

In [13]:
# This time we group by payment_type and pickup_interval before computing the mean as in task 8
avg_fare_by_interval = df.groupby(['payment_type', 'pickup_interval'])[['fare_amount']].mean()

10. For each payment type, determine the interval when the average fare amount is maximum


In [14]:
# When you use groupby along with idxmax, it returns the index of the maximum value for each group, 
# but it does not directly print the IDs. Instead, it returns a DataFrame or Series with the indices 
# where the maximum values occur.
max_avg_fare_interval = avg_fare_by_interval.groupby('payment_type').idxmax()
max_avg_fare_interval

,fare_amount
payment_type,
1.0,"(1.0, 05:00 - 05:30)"
2.0,"(2.0, 05:00 - 05:30)"
3.0,"(3.0, 07:00 - 07:30)"
4.0,"(4.0, 05:00 - 05:30)"
5.0,"(5.0, 17:30 - 18:00)"


11. For each payment type, determine the interval when the overall ratio between the tip and the fare amounts is maximum

In [15]:
# Firstly we calculate the ratio between tip and fare amounts and store it in a new column `tip_fare_ratio`
df['tip_fare_ratio'] = df['tip_amount'] / df['fare_amount']

# Then we group the data by `payment_type` and `pickup_interval` and for each group, calculate the average 
# tip-to-fare ratio.
# Lastly, for each `payment_type`, find the `pickup_interval` with the highest average `tip_fare_ratio`
df.groupby(['payment_type', 'pickup_interval'])[['tip_fare_ratio']].mean().groupby('payment_type').idxmax()

# 5.0 is NaN because the trip_amount and fare_amount is 0

,tip_fare_ratio
payment_type,
1.0,"(1.0, 00:00 - 00:30)"
2.0,"(2.0, 09:00 - 09:30)"
3.0,"(3.0, 21:00 - 21:30)"
4.0,"(4.0, 10:00 - 10:30)"
5.0,NaN


12. Find the location with the highest average fare amount

In [16]:
# Group by PULocationID and DOLocationID and calculate the mean fare amount
pick_up_avg_fare = df.groupby(['PULocationID'])['fare_amount'].mean()
drop_off_avg_fare = df.groupby(['DOLocationID'])['fare_amount'].mean()

# To show the results:
print(f"For pickup the location with higehest average is: {pick_up_avg_fare.idxmax()} with avg fare {pick_up_avg_fare.max()}")
print(f"For dropoff the location with higehest average is: {drop_off_avg_fare.idxmax()} with avg fare {drop_off_avg_fare.max()}")

For pickup the location with higehest average is: 204 with avg fare 107.0
For dropoff the location with higehest average is: 44 with avg fare 86.77377049180328


13. Build a new dataframe (called common) where, for each pickup location we keep all trips to the 5 most common destinations (i.e. each pickup location can have different common destinations).

In [17]:
# We get the 5 most common destinations for each pickup location first
top_5_destinations = df.groupby('PULocationID')['DOLocationID'].value_counts().groupby(level=0).head(5).reset_index(name='count')
top_5_destinations.head(10)

,PULocationID,DOLocationID,count
0,1,1,636
1,1,264,105
2,1,265,4
3,1,138,2
4,1,50,1
5,2,56,1
6,2,95,1
7,2,201,1
8,3,226,22
9,3,3,5


In [18]:
# Then we create the 'common' df
common = df.merge(top_5_destinations[['PULocationID', 'DOLocationID']], on=['PULocationID', 'DOLocationID'])
common.head(10)


# Remove all columns that are not VendorId, tpep_pickup_datetime, tpep_dropoff_datetime, PULocationID or DOLocationID, pickup_interval



,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration,pickup_interval,tip_fare_ratio
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,...,3.0,0.5,1.47,0.0,0.3,11.27,2.5,0 days 00:04:48,00:00 - 00:30,0.245000
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,N,239,238,1.0,...,3.0,0.5,1.50,0.0,0.3,12.30,2.5,0 days 00:07:25,00:30 - 01:00,0.214286
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,N,238,238,1.0,...,3.0,0.5,1.00,0.0,0.3,10.80,2.5,0 days 00:06:11,00:30 - 01:00,0.166667
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,N,238,151,1.0,...,0.5,0.5,1.36,0.0,0.3,8.16,0.0,0 days 00:04:51,00:30 - 01:00,0.247273
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,N,193,193,2.0,...,0.5,0.5,0.00,0.0,0.3,4.80,0.0,0 days 00:02:18,00:00 - 00:30,0.000000
5,2.0,2020-01-01 00:09:44,2020-01-01 00:10:37,1.0,0.03,1.0,N,7,193,2.0,...,0.5,0.5,0.00,0.0,0.3,3.80,0.0,0 days 00:00:53,00:00 - 00:30,0.000000
6,2.0,2020-01-01 00:39:25,2020-01-01 00:39:29,1.0,0.00,1.0,N,193,193,1.0,...,0.5,0.5,0.01,0.0,0.3,3.81,0.0,0 days 00:00:04,00:30 - 01:00,0.004000
7,2.0,2019-12-18 15:27:49,2019-12-18 15:28:59,1.0,0.00,5.0,N,193,193,1.0,...,0.0,0.0,0.00,0.0,0.3,2.81,2.5,0 days 00:01:10,15:00 - 15:30,0.000000
8,2.0,2019-12-18 15:30:35,2019-12-18 15:31:35,4.0,0.00,1.0,N,193,193,1.0,...,0.5,0.5,0.00,0.0,0.3,6.30,2.5,0 days 00:01:00,15:30 - 16:00,0.000000
9,1.0,2020-01-01 00:29:01,2020-01-01 00:40:28,2.0,0.70,1.0,N,246,48,1.0,...,3.0,0.5,2.35,0.0,0.3,14.15,2.5,0 days 00:11:27,00:00 - 00:30,0.293750


In [19]:
dfCopy = df[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'pickup_interval']]
dfCopy.sort_values(['VendorID', 'tpep_pickup_datetime']).copy()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,pickup_interval
8483,1.0,2020-01-01 00:00:00,2020-01-01 00:13:03,68,170,00:00 - 00:30
3083,1.0,2020-01-01 00:00:03,2020-01-01 00:13:04,79,162,00:00 - 00:30
5760,1.0,2020-01-01 00:00:05,2020-01-01 00:26:30,264,68,00:00 - 00:30
5050,1.0,2020-01-01 00:00:07,2020-01-01 00:03:26,75,75,00:00 - 00:30
2555,1.0,2020-01-01 00:00:25,2020-01-01 00:05:59,145,179,00:00 - 00:30
...,...,...,...,...,...,...
4269480,2.0,2020-07-10 11:34:11,2020-07-10 11:42:41,236,262,11:30 - 12:00
4282277,2.0,2020-07-31 18:50:41,2020-07-31 18:54:12,236,43,18:30 - 19:00
275044,2.0,2021-01-02 00:22:00,2021-01-02 00:36:50,142,161,00:00 - 00:30
275045,2.0,2021-01-02 00:44:08,2021-01-02 00:58:56,170,148,00:30 - 01:00


14. On the common dataframe, for each payment type and each interval, determine the average fare amount

In [20]:
# As with the other previous tasks, we solve this task using .mean() and .groupby() 
common_avg_fare_by_interval = common.groupby(['payment_type', 'pickup_interval'])[['fare_amount']].mean()

15. Compute the difference of the average fare amount computed in the previous point with those computed at point 9.

In [21]:
fare_diff = common_avg_fare_by_interval - avg_fare_by_interval
fare_diff

fare_amount
payment_type pickup_interval             
1.0          00:00 - 00:30      -5.280185
             00:30 - 01:00      -4.781551
             01:00 - 01:30      -4.328460
             01:30 - 02:00      -4.331167
             02:00 - 02:30      -4.062679
...                                   ...
4.0          22:00 - 22:30      -0.233207
             22:30 - 23:00       0.223146
             23:00 - 23:30      -2.217294
             23:30 - 24:00      -1.660740
5.0          17:30 - 18:00       0.000000

[193 rows x 1 columns]

16. Compute the ratio between the differences computed in the previous point and those computed in point 9. Note: you have to compute a ratio for each pair (payment type, interval).

In [22]:
ratio_diff = fare_diff / avg_fare_by_interval
ratio_diff

fare_amount
payment_type pickup_interval             
1.0          00:00 - 00:30      -0.380715
             00:30 - 01:00      -0.354919
             01:00 - 01:30      -0.337512
             01:30 - 02:00      -0.350475
             02:00 - 02:30      -0.338314
...                                   ...
4.0          22:00 - 22:30      -0.152092
             22:30 - 23:00      -0.283508
             23:00 - 23:30       6.312096
             23:30 - 24:00       0.604250
5.0          17:30 - 18:00            NaN

[193 rows x 1 columns]

17. Build chains of trips. Two trips are consecutive in a chain if (a) they have the same VendorID, (b) the pickup location of the second trip is also the dropoff location of the first trip, (c) the pickup time of the second trip is after the dropoff time of the first trip, and (d) the pickup time of the second trip is at most 2 minutes later than the dropoff time of the first trip.

In [23]:
# Sort by vendor and pickup time
df = df.sort_values(['VendorID', 'tpep_pickup_datetime']).copy()

In [24]:
final = df.head(10000)

current_chain = 1

#Add a chain column to the dataframe
final['chain'] = None

# Group by VendorID and process each vendor separately
for vendor_id, vendor_group in final.groupby('VendorID'):
    vendor_group = vendor_group.sort_values('tpep_pickup_datetime').copy()
    
    while len(vendor_group) > 0:
        current = vendor_group.iloc[0]
                    
        final.loc[current.name, 'chain'] = current_chain
        
        # Remove current trip 
        vendor_group = vendor_group.iloc[1:]
        
        while True:
            # Find successors
            mask = (
                (vendor_group['PULocationID'] == current['DOLocationID']) &
                (vendor_group['tpep_pickup_datetime'] > current['tpep_dropoff_datetime']) &
                (vendor_group['tpep_pickup_datetime'] <= 
                 current['tpep_dropoff_datetime'] + pd.Timedelta(minutes=2))
            )
            possible_successors = vendor_group[mask]
            
            # If no successors found, break inner loop to start new chain
            if len(possible_successors) == 0:
                current_chain += 1
                break
            
            successor = possible_successors.iloc[0]
            
            final.loc[successor.name, 'chain'] = current_chain
            
            # Remove from working group
            vendor_group = vendor_group[vendor_group.index != successor.name]
            
            current = successor
            
final[final['chain'] == 1]

C:\Users\1\AppData\Local\Temp\ipykernel_31360\648109053.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['chain'] = None


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration,pickup_interval,tip_fare_ratio,chain
8483,1.0,2020-01-01 00:00:00,2020-01-01 00:13:03,1.0,2.2,1.0,N,68,170,1.0,...,0.5,2.85,0.0,0.3,17.15,2.5,0 days 00:13:03,00:00 - 00:30,0.271429,1
3589,1.0,2020-01-01 00:13:28,2020-01-01 00:31:48,1.0,3.8,1.0,N,170,239,2.0,...,0.5,0.00,0.0,0.3,19.30,2.5,0 days 00:18:20,00:00 - 00:30,0.000000,1
7439,1.0,2020-01-01 00:31:54,2020-01-01 01:42:07,2.0,8.0,1.0,N,239,25,1.0,...,0.5,9.35,0.0,0.3,56.15,2.5,0 days 01:10:13,00:30 - 01:00,0.217442,1
19216,1.0,2020-01-01 01:43:32,2020-01-01 01:43:32,0.0,0.0,1.0,Y,25,264,2.0,...,0.5,0.00,0.0,0.3,6.80,0.0,0 days 00:00:00,01:30 - 02:00,0.000000,1
14079,1.0,2020-01-01 01:44:43,2020-01-01 02:08:36,1.0,4.4,1.0,N,264,264,1.0,...,0.5,5.75,0.0,0.3,25.05,0.0,0 days 00:23:53,01:30 - 02:00,0.319444,1


In [ ]:
to_be_worked_on = df.head(10000)
df['chaij'] = None

# Sort by VendorID and both datetime columns
to_be_worked_on_left = to_be_worked_on.sort_values(['VendorID', 'tpep_dropoff_datetime'])
to_be_worked_on_right = to_be_worked_on.sort_values(['VendorID', 'tpep_pickup_datetime'])

# Perform the merge_asof with the properly sorted DataFrames
to_be_worked_on = pd.merge_asof(
    to_be_worked_on_left, 
    to_be_worked_on_right,
    by='VendorID',
    left_on='tpep_dropoff_datetime',
    right_on='tpep_pickup_datetime',
    suffixes=('', '_next')
)

# Rest of your code remains the same
to_be_worked_on = to_be_worked_on[
    (to_be_worked_on['PULocationID_next'] == to_be_worked_on['DOLocationID']) &
    (to_be_worked_on['tpep_pickup_datetime_next'] > to_be_worked_on['tpep_dropoff_datetime']) &
    (to_be_worked_on['tpep_pickup_datetime_next'] <= to_be_worked_on['tpep_dropoff_datetime'] + pd.Timedelta(minutes=2))
]
# Initialize chain IDs
to_be_worked_on['chain'] = None
current_chain = 1

# Assign chain IDs iteratively
for idx, row in to_be_worked_on.iterrows():
    if pd.isnull(to_be_worked_on.loc[idx, 'chain']):  # start a new chain if chain ID is not assigned
        to_be_worked_on.loc[idx, 'chain'] = current_chain
        next_idx = idx
        while True:
            # Get the next trip in the chain
            next_row = to_be_worked_on[(to_be_worked_on['VendorID'] == row['VendorID']) &
                          (to_be_worked_on['PULocationID'] == row['DOLocationID']) &
                          (to_be_worked_on['tpep_pickup_datetime'] > row['tpep_dropoff_datetime']) &
                          (to_be_worked_on['tpep_pickup_datetime'] <= row['tpep_dropoff_datetime'] + pd.Timedelta(minutes=2))]
            if next_row.empty:
                break
            next_idx = next_row.index[0]
            df.loc[next_idx, 'chain'] = current_chain
            row = next_row.iloc[0]
        current_chain += 1
        
        
#to_be_worked_on[to_be_worked_on['chain'] == 1]
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration,pickup_interval,tip_fare_ratio
8483,1.0,2020-01-01 00:00:00,2020-01-01 00:13:03,1.0,2.20,1.0,N,68,170,1.0,...,3.0,0.5,2.85,0.0,0.3,17.15,2.5,0 days 00:13:03,00:00 - 00:30,0.271429
3083,1.0,2020-01-01 00:00:03,2020-01-01 00:13:04,2.0,3.00,1.0,N,79,162,1.0,...,3.0,0.5,2.00,0.0,0.3,17.30,2.5,0 days 00:13:01,00:00 - 00:30,0.173913
5760,1.0,2020-01-01 00:00:05,2020-01-01 00:26:30,0.0,3.40,1.0,N,264,68,1.0,...,3.0,0.5,4.35,0.0,0.3,26.15,2.5,0 days 00:26:25,00:00 - 00:30,0.241667
5050,1.0,2020-01-01 00:00:07,2020-01-01 00:03:26,3.0,0.60,1.0,N,75,75,2.0,...,0.5,0.5,0.00,0.0,0.3,5.80,0.0,0 days 00:03:19,00:00 - 00:30,0.000000
2555,1.0,2020-01-01 00:00:25,2020-01-01 00:05:59,1.0,1.70,1.0,N,145,179,2.0,...,0.5,0.5,0.00,0.0,0.3,8.30,0.0,0 days 00:05:34,00:00 - 00:30,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4269480,2.0,2020-07-10 11:34:11,2020-07-10 11:42:41,1.0,1.07,1.0,N,236,262,1.0,...,1.0,0.5,2.26,0.0,0.3,13.56,2.5,0 days 00:08:30,11:30 - 12:00,0.322857
4282277,2.0,2020-07-31 18:50:41,2020-07-31 18:54:12,1.0,0.72,1.0,N,236,43,2.0,...,1.0,0.5,0.00,0.0,0.3,9.30,2.5,0 days 00:03:31,18:30 - 19:00,0.000000
275044,2.0,2021-01-02 00:22:00,2021-01-02 00:36:50,1.0,1.56,1.0,N,142,161,2.0,...,1.0,0.5,0.00,0.0,0.3,14.80,2.5,0 days 00:14:50,00:00 - 00:30,0.000000
275045,2.0,2021-01-02 00:44:08,2021-01-02 00:58:56,1.0,2.32,1.0,N,170,148,2.0,...,1.0,0.5,0.00,0.0,0.3,15.80,2.5,0 days 00:14:48,00:30 - 01:00,0.000000


chain_id = 0

# Group by VendorID first
for vendor_id, vendor_group in df.groupby('VendorID'):
    # Create shift operations for comparison
    next_pu_loc = vendor_group['PULocationID'].shift(-1)
    curr_do_loc = vendor_group['DOLocationID']
    next_pu_time = vendor_group['tpep_pickup_datetime'].shift(-1)
    curr_do_time = vendor_group['tpep_dropoff_datetime']
    
    # Calculate time difference in minutes
    time_diff = (next_pu_time - curr_do_time).dt.total_seconds() / 60
    
    # Create panda Series of potential chains
    chain_links = (
        (next_pu_loc == curr_do_loc) & 
        (time_diff > 0) & 
        (time_diff <= 2)
    )
    
    # Create Process each continuous sequence
    current_chain = []
    
    for idx, is_chain in chain_links.items():
        if not current_chain:  # Start new chain
            current_chain = [idx]
        
        if is_chain:  # Add to current chain
            current_chain.append(vendor_group.index[vendor_group.index.get_loc(idx) + 1])
        else:  # End current chain
            if len(current_chain) > 0:
                df.loc[current_chain, 'chain'] = chain_id
                chain_id += 1
            current_chain = []
    
    # Handle last chain if exists
    if current_chain:
        df.loc[current_chain, 'chain'] = chain_id
        chain_id += 1

In [35]:
# Select everything in df that is VendorId , pickup time, drop off time, pick up location, drop off location and pickup interval
df[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'pickup_interval']]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,pickup_interval
8483,1.0,2020-01-01 00:00:00,2020-01-01 00:13:03,68,170,00:00 - 00:30
3083,1.0,2020-01-01 00:00:03,2020-01-01 00:13:04,79,162,00:00 - 00:30
5760,1.0,2020-01-01 00:00:05,2020-01-01 00:26:30,264,68,00:00 - 00:30
5050,1.0,2020-01-01 00:00:07,2020-01-01 00:03:26,75,75,00:00 - 00:30
2555,1.0,2020-01-01 00:00:25,2020-01-01 00:05:59,145,179,00:00 - 00:30
...,...,...,...,...,...,...
4269480,2.0,2020-07-10 11:34:11,2020-07-10 11:42:41,236,262,11:30 - 12:00
4282277,2.0,2020-07-31 18:50:41,2020-07-31 18:54:12,236,43,18:30 - 19:00
275044,2.0,2021-01-02 00:22:00,2021-01-02 00:36:50,142,161,00:00 - 00:30
275045,2.0,2021-01-02 00:44:08,2021-01-02 00:58:56,170,148,00:30 - 01:00


# Recursive function to test one index to see its chain
def get_possible_successor(remaining, curr_trip_idx, previous_in_chain, chain_id, final):
    curr_trip = remaining.iloc[curr_trip_idx]
    
    mask = (
        (remaining['VendorID'] == curr_trip['VendorID']) &
        (remaining['PULocationID'] == curr_trip['DOLocationID']) &
        (remaining['tpep_pickup_datetime'] > curr_trip['tpep_dropoff_datetime']) &
        (remaining['tpep_pickup_datetime'] <= 
         curr_trip['tpep_dropoff_datetime'] + pd.Timedelta(minutes=2))
    )
    
    possible_successors = remaining[mask]
    
    original_idx = remaining.index[curr_trip_idx]
    
    if previous_in_chain:
        final.loc[original_idx, 'chain'] = chain_id
    
    remaining = remaining.drop(index=original_idx)
    
    # If no successors then start new chain
    if len(possible_successors) == 0:
        if len(remaining) > 0:
            if not previous_in_chain:
                chain_id += 1
            final.loc[original_idx, 'chain'] = chain_id
            #get_possible_successor(remaining, 0, False, chain_id, final)
        return chain_id
    
    # Start new chain if previous wasn't in chain
    if not previous_in_chain:
        chain_id += 1
        final.loc[original_idx, 'chain'] = chain_id
    
    # Continue with first successor
    successor_idx = possible_successors.index[0]
    next_trip_idx = remaining.index.get_loc(successor_idx)
    return get_possible_successor(remaining, next_trip_idx, True, chain_id, final)


remaining = df.copy()
final = df.copy()

#Sort by vendor and pickup time
remaining = remaining.sort_values(['VendorID', 'tpep_pickup_datetime'])


index = 0

get_possible_successor(remaining, index, False, 0, final)
final[final['chain'] == 1].head(10)

Hint: Add a column chain to the dataset. A chain can have more than two trips.